In [16]:
import h5py
import numpy as np

In [17]:
with h5py.File('data.h5', 'r') as hf:
    X = hf['EEG_values'][:]             #Samples tensor
    y = hf['target_values'][:]          #Targets matrix

In [18]:
channels_order = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']

In [19]:
from PCA_MTS import PCA_MTS

In [20]:
New_space = PCA_MTS(X)

Pls make sure that Input tensor X as shape of M: Samples , ni: length of time sereis , m: Numper of features 


In [21]:
eig_vals, eig_vecs , junk , junk = New_space.eigens()

In [22]:
Space = eig_vecs[:,0:12]

In [23]:
X     = np.dot(X,Space)

In [25]:
X

array([[[-15484.403   ,   1231.5172  ,   1461.9069  , ...,
            -59.706024,    -87.15874 ,     70.763275],
        [-15504.95    ,   1240.0381  ,   1458.2832  , ...,
            -76.20766 ,    -81.53526 ,     59.945343],
        [-15562.651   ,   1234.7153  ,   1457.6438  , ...,
            -78.71303 ,    -84.00462 ,     49.07196 ],
        ...,
        [-15514.518   ,   1220.0203  ,   1442.5892  , ...,
            -74.87509 ,    -93.56128 ,     68.639435],
        [-15526.048   ,   1230.3474  ,   1433.5319  , ...,
            -68.85254 ,    -87.98866 ,     69.64087 ],
        [-15514.583   ,   1219.5206  ,   1425.3303  , ...,
            -63.276276,    -85.58206 ,     73.097534]],

       [[-15512.712   ,   1213.929   ,   1432.7726  , ...,
            -62.82567 ,    -91.51369 ,     67.97757 ],
        [-15496.02    ,   1228.5571  ,   1445.66    , ...,
            -59.59407 ,    -96.81656 ,     62.185608],
        [-15479.499   ,   1235.6769  ,   1441.7114  , ...,
            -5